# VGG-16 pre-trained network example

#### Imports

In [ ]:
%matplotlib inline

import os

import numpy as np

from matplotlib import pyplot as plt

import sklearn.cross_validation

import lasagne
import theano.tensor as T
from britefury_lasagne import basic_dnn
from britefury_lasagne.pretrained import imagenet_vgg

### Load an example image to classify

Load an image containing a peacock, and extract an 896x896 (4 x 224, where 224 is the image size expected by the network) block containing the peacock.

In [ ]:
IMAGE_PATH = os.path.join('images', 'P1013781.JPG')
# Extract a 896 x 896 block surrounding the peacock
img = plt.imread(IMAGE_PATH)[1800:2696,652:1548]
plt.imshow(img)
plt.show()

### Load the VGG-16 network

Invoke `VGG16Model.load()` to - if necessary - download the network weights, load them and construct the model:

In [ ]:
vgg16 = imagenet_vgg.VGG16Model.load()

### Show the layer names, types and output shapes

In [4]:
for name, layer in vgg16.network.items():
    print('{: <22} type {: <21}: output_shape={}'.format(name, type(layer).__name__, layer.output_shape))

input                  type InputLayer           : output_shape=(None, 3, 224, 224)
conv1_1                type Conv2DLayer          : output_shape=(None, 64, 224, 224)
conv1_2                type Conv2DLayer          : output_shape=(None, 64, 224, 224)
pool1                  type Pool2DLayer          : output_shape=(None, 64, 112, 112)
conv2_1                type Conv2DLayer          : output_shape=(None, 128, 112, 112)
conv2_2                type Conv2DLayer          : output_shape=(None, 128, 112, 112)
pool2                  type Pool2DLayer          : output_shape=(None, 128, 56, 56)
conv3_1                type Conv2DLayer          : output_shape=(None, 256, 56, 56)
conv3_2                type Conv2DLayer          : output_shape=(None, 256, 56, 56)
conv3_3                type Conv2DLayer          : output_shape=(None, 256, 56, 56)
pool3                  type Pool2DLayer          : output_shape=(None, 256, 28, 28)
conv4_1                type Conv2DLayer          : output_shape=(None

#### Build the classifier

In [ ]:
# The network builder function returns the last layer of the pre-trained model, nothing more
def build_network(input_vars):
    return vgg16.final_layer

# Build the image classifier for the given model builder
print('Building network')
# We can't use the `simple_classifier` function, as we have to supply the input variables, so use
# the `classifier` function. Also, tell the `classifier` function that the model includes
# the softmax non-linearity in the final layer, so that it does not add a second one
clf = basic_dnn.classifier(None, build_network, includes_softmax=True)

### Use the classifier to predict the class of the image

In [ ]:
# Convert the image for use with VGG-16; this will scale and crop to 224x224,
# flip RGB to BGR channel order, subtract the mean value, etc...
raw_img, img_for_vgg = vgg16.prepare_image(img)

# Predict probabilities for the image
img_pred_prob = clf.predict([img_for_vgg])[0]

# Print the predicted probability shape
print(img_pred_prob.shape)

# Find the predicted class and display it
pred_cls = np.argmax(img_pred_prob, axis=1)
print('Predicted class index {} with probability {:.2%}, named "{}"'.format(
    pred_cls[0], img_pred_prob[0, pred_cls[0]], vgg16.class_names[pred_cls[0]]))